# USER SIMILARITY BASED RECOMMENDER

Development of a user-based machine learning to recommend books that user still don't read but users with similar caracteres already read and liked them.

The used datasets are abled to download on [link]()

In [ ]:
# libraries
import numpy as np
import pandas as pd

import ast

import nltk
from gensim.utils import tokenize
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors

import warnings
warnings.filterwarnings(action = 'ignore')

In [ ]:
# loading data
movies = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Recommendation Systems/Datasets/tmdb_5000_movies.csv')
credits = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Recommendation Systems/Datasets/tmdb_5000_credits.csv')

In [ ]:
movies.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


In [ ]:
credits.head(3)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


In [ ]:
movies = movies.merge(credits, on='title')
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

## Pré-Processing

In [ ]:
movies = movies[['id', 'title', 'overview', 'genres', 'keywords','cast', 'crew']]

In [ ]:
def get_name(element):
  genre_list = []
  for i in ast.literal_eval(element):
    genre_list.append(i['name'])
  return genre_list

movies['genres'] = movies['genres'].apply(get_name)
movies['genres'] = movies['genres'].apply(lambda x: [i.replace(" ","") for i in x])
movies.head(2)

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, ScienceFiction]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [ ]:
def tokenize_text(text):
  return list(tokenize(str(text)))

movies['overview'] = movies['overview'].apply(tokenize_text)

movies.head()

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, nd, century, a, paraplegic, Marine, ...","[Action, Adventure, Fantasy, ScienceFiction]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa, long, believed, to, be, de...","[Adventure, Fantasy, Action]","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[A, cryptic, message, from, Bond, s, past, sen...","[Action, Adventure, Crime]","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[John, Carter, is, a, war, weary, former, mili...","[Action, Adventure, ScienceFiction]","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [ ]:
movies['keywords'] = movies['keywords'].apply(get_name)
movies['keywords'] = movies['keywords'].apply(lambda x: [i.replace(" ","") for i in x])
movies.head(2)

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, nd, century, a, paraplegic, Marine, ...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa, long, believed, to, be, de...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [ ]:
def get_actors(element):
  actors_list = []
  count = 0
  for i in ast.literal_eval(element):
    if count != 3:
      actors_list.append(i['name'])
      count += 1
    else:
      break
  return actors_list

movies['main_actors'] = movies['cast'].apply(get_actors)
movies['main_actors'] = movies['main_actors'].apply(lambda x: [i.replace(" ","") for i in x])
movies.drop(labels=['cast'], axis=1, inplace=True)
movies.head(2)

,id,title,overview,genres,keywords,crew,main_actors
0,19995,Avatar,"[In, the, nd, century, a, paraplegic, Marine, ...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[SamWorthington, ZoeSaldana, SigourneyWeaver]"
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa, long, believed, to, be, de...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]"


In [ ]:
def get_directors(element):
  directors_list = []
  for i in ast.literal_eval(element):
    if i['job'] == 'Director':
      directors_list.append(i['name'])
  return directors_list

movies['directors'] = movies['crew'].apply(get_directors)
movies['directors'] = movies['directors'].apply(lambda x: [i.replace(" ","") for i in x])
movies.drop(labels=['crew'], axis=1, inplace=True)
movies.head(2)

,id,title,overview,genres,keywords,main_actors,directors
0,19995,Avatar,"[In, the, nd, century, a, paraplegic, Marine, ...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa, long, believed, to, be, de...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]


In [ ]:
data = movies[['id', 'title']]
data['tags'] = movies['overview'] +	movies['genres'] +	movies['keywords'] +	movies['main_actors'] +	movies['directors']
data['tags'] = data['tags'].apply(lambda x:" ".join(x))
data['tags'] = data['tags'].apply(lambda x: x.lower())

data.head()

,id,title,tags
0,19995,Avatar,in the nd century a paraplegic marine is dispa...
1,285,Pirates of the Caribbean: At World's End,captain barbossa long believed to be dead has ...
2,206647,Spectre,a cryptic message from bond s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,john carter is a war weary former military cap...


In [ ]:
ps = PorterStemmer()

def stem(text):
  word = []
  for i in text.split():
    word.append(ps.stem(i))

  return " ".join(word)

data['tags'] = data['tags'].apply(stem)

In [ ]:
cv = CountVectorizer(max_features=5000, stop_words='english')
samples = cv.fit_transform(data['tags']).toarray()

In [ ]:
cv.get_feature_names()

['18thcenturi',
 '1930',
 '1940',
 '1950',
 '1960',
 '1970',
 '1980',
 '19thcenturi',
 '3d',
 'aaron',
 'aaroneckhart',
 'abandon',
 'abbi',
 'abduct',
 'abigailbreslin',
 'abil',
 'abl',
 'aboard',
 'aborigin',
 'abort',
 'abov',
 'abroad',
 'absenc',
 'absolut',
 'absurd',
 'abus',
 'academ',
 'academi',
 'accept',
 'access',
 'accid',
 'accident',
 'acclaim',
 'accompani',
 'accomplish',
 'account',
 'accus',
 'ace',
 'achiev',
 'acquaint',
 'act',
 'action',
 'actionhero',
 'activ',
 'activist',
 'actor',
 'actress',
 'actual',
 'ad',
 'adam',
 'adamsandl',
 'adamshankman',
 'adapt',
 'add',
 'addict',
 'addit',
 'adjust',
 'admir',
 'admit',
 'adolesc',
 'adopt',
 'ador',
 'adrienbrodi',
 'adult',
 'adultanim',
 'adulteri',
 'adulthood',
 'advanc',
 'advantag',
 'adventur',
 'advertis',
 'advic',
 'advis',
 'affair',
 'affect',
 'afghanistan',
 'africa',
 'african',
 'africanamerican',
 'aftercreditssting',
 'afterlif',
 'aftermath',
 'age',
 'agediffer',
 'agenc',
 'agenda',
 'ag

## Machine Learning

In [ ]:
similarity = cosine_similarity(samples)

def recommender_machine(movie):

  movie_index = data[data['title']==movie].index[0]
  dist = similarity[movie_index]
  movies_list = sorted(list(enumerate(dist)), reverse=True, key= lambda x: x[1])[1:5]

  for i in movies_list:
    print(data.iloc[i[0]].title)

## Recommending to an user

In [ ]:
recommender_machine("Pirates of the Caribbean: At World's End")

Pirates of the Caribbean: Dead Man's Chest
Pirates of the Caribbean: The Curse of the Black Pearl
Pirates of the Caribbean: On Stranger Tides
Life of Pi


In [ ]:
import pickle as pkl
pkl.dump(data, open('/content/drive/MyDrive/Colab Notebooks/Recommendation Systems/movies.pkl', 'wb'))
pkl.dump(data.to_dict(), open('/content/drive/MyDrive/Colab Notebooks/Recommendation Systems/movies_dict.pkl', 'wb'))
pkl.dump(similarity, open('/content/drive/MyDrive/Colab Notebooks/Recommendation Systems/similarity.pkl', 'wb'))

# popularity

In [ ]:
movies = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Recommendation Systems/Datasets/tmdb_5000_movies.csv')
movies['genres'] = movies['genres'].apply(get_name)
movies['genres'] = movies['genres'].apply(lambda x: [i.replace(" ","") for i in x])
movies_by_popularity = movies[['id','title', 'popularity', 'genres']]
movies_by_popularity.sort_values(by='popularity', ascending=False).head()

,id,title,popularity,genres
546,211672,Minions,875.581305,"[Family, Animation, Adventure, Comedy]"
95,157336,Interstellar,724.247784,"[Adventure, Drama, ScienceFiction]"
788,293660,Deadpool,514.569956,"[Action, Adventure, Comedy]"
94,118340,Guardians of the Galaxy,481.098624,"[Action, ScienceFiction, Adventure]"
127,76341,Mad Max: Fury Road,434.278564,"[Action, Adventure, ScienceFiction, Thriller]"


In [ ]:
pkl.dump(movies_by_popularity.to_dict(), open('/content/drive/MyDrive/Colab Notebooks/Recommendation Systems/popular_movies.pkl', 'wb'))